In [1]:
# Importing requird python packages
# Path to hsmpy package
HSMPY_PATH = r'C:\Users\MR068144\Downloads\Python Scripts'
import sys, os, arcpy, csv, json, math
sys.path.append(HSMPY_PATH)
import hsmpy3
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import datetime
from IPython.display import Image
from IPython.core.display import HTML 
import subprocess 
import webbrowser
import time
import copy
import shutil
from time import gmtime, strftime
from ipywidgets import FloatProgress
print(strftime("%Y-%m-%d %H:%M:%S"))

2018-08-13 10:10:34


In [2]:
# Defining Parameteres
print(strftime("%Y-%m-%d %H:%M:%S"))
Years = range(2012,2017)
MaxOpenProcesses = 15
Project_DIR  = r'\\CHCFPP01\Proj\ILDOT\650511SAFETYPROGRAM\4_WorkData\WO00_OtherStates\NewMexico\3_WorkData\7_NetworkScreening'
ReceivedDir  = Project_DIR + '\\2.Received_Data'
Project_GDB  = Project_DIR + '\\5.NM_NS_GDB.gdb'

Src_Loc  = ReceivedDir + '\\crash_asof_11jun2018.csv'
Src_Unit = ReceivedDir + '\\vehicle_asof_11jun2018.csv'
Src_Occ  = ReceivedDir + '\\occupant_asof_11jun2018.csv'

HPMS105 = {year:ReceivedDir + '\\v105\\jg_request.gdb\\HPMS{}Data'.format(year) for year in Years}
HPMS10  = {year:ReceivedDir + '\\v10\\jg_request.gdb\\HPMS{}Data'.format(year) for year in Years}

Routes  = {year:Project_GDB + '\\Routes_{}'.format(2016) for year in Years}
AttTabs = {year:Project_GDB + '\\SegAtt_{}'.format(year) for year in Years}
Segs    = {year:Project_GDB + '\\Seg_{}'.format(year) for year in Years}
CrashData = {year:Project_GDB + '\\Crash_{}'.format(year) for year in Years}

UTM13N = arcpy.SpatialReference(26913)
NAD83_NM = arcpy.SpatialReference(6529)

RouteID = "ROUTE_ID"
BEG_STA = 'BEG_POINT'
END_STA = 'END_POINT'
Fields = ['F_SYSTEM','THROUGH_LANES','FACILITY_TYPE','AADT','MEDIAN_TYPE','SPEED_LIMIT','LANE_WIDTH']

BridgeUnder = ReceivedDir + '\\v10\\jg_request.gdb\\BridgeUnder'
BridgeLRS   = ReceivedDir + '\\v10\\jg_request.gdb\\BridgeLRS'
Signals     = ReceivedDir + '\\v10\\jg_request.gdb\\SignalizedIntersections'

2018-08-13 10:10:39


In [3]:
def OppositeDirID(InpID):
    out = InpID[:-2]
    ext = InpID[-1]
    if ext == 'P':
        return('{}-M'.format(out))
    if ext == 'M':
        return('{}-P'.format(out))
def NewBMPEMP(RID,BMP,EMP,NextRID,NextBMP,PrevRID,PrevEMP):
        NewBMP = BMP
        NewEMP = EMP
        if NextRID == RID:
            if NextBMP<EMP:
                NewEMP = np.mean([NextBMP,EMP])
        if PrevRID==RID:
            if PrevEMP>BMP:
                NewBMP = np.mean([PrevEMP,BMP])
        return([NewBMP,NewEMP])
def FindIntPG(Type_Major, Legs, Signal):
    if Type_Major=='R2U':
        if Signal == 0:
            if Legs==3:
                return('R3ST')
            if Legs == 4:
                return('R4ST')
        if Signal == 1:
            if Legs==3:
                return('R3SG')
            if Legs == 4:
                return('R4SG')
    if Type_Major in ['R4U','R4D']:
        if Signal == 0:
            if Legs==3:
                return('RM3ST')
            if Legs == 4:
                return('RM4ST')
        if Signal == 1:
            if Legs==3:
                return('RM3SG')
            if Legs == 4:
                return('RM4SG')
    if Type_Major in ['U2U','U4U','U4D']:
        if Signal == 0:
            if Legs==3:
                return('U3ST')
            if Legs == 4:
                return('U4ST')
        if Signal == 1:
            if Legs==3:
                return('U3SG')
            if Legs == 4:
                return('U4SG')

In [4]:
Att_Dict = {year:hsmpy3.common.AttributeTabletoDF(AttTabs[year]) for year in Years}
for year in Years:
    
    Att_Dict[year] = Att_Dict[year].sort_values(by=['ROUTE_ID','BEG_POINT'])
    Att_Dict[year]['Len'] = Att_Dict[year].END_POINT-Att_Dict[year].BEG_POINT
    Att_Dict[year] = Att_Dict[year][(Att_Dict[year].Len>0)]
    Att_Dict[year]['SurfWid'] = 24.0
    for i,r in Att_Dict[year].iterrows():
        nl = 2
        try:
            nl = int(r.THROUGH_LANES)
        except:
            pass
        lw = 12.0
        try:
            lw = float(r.LANE_WIDTH)
        except:
            pass
        mdw = {1:0,2:4,3:4,4:6,5:6,6:6,7:6,0:0}
        md = 1
        try:
            md = int(r.MEDIAN_TYPE)
        except:
            pass
        surfwid = max(24,nl*lw+mdw[md])
        Att_Dict[year].set_value(i,'SurfWid',surfwid)

        aadt = r.AADT
        try:
            aadt = int(aadt)
        except:
            aadt = 0
        Att_Dict[year].set_value(i,'AADT',aadt)

In [5]:
Route_Dict = {year:hsmpy3.common.FCtoDF(Routes[year],True) for year in Years}

In [6]:
Int_DF = hsmpy3.common.FCtoDF(Project_GDB+'\\CreateInt_Final',True)

In [7]:
for year in Years:
    EventTable = hsmpy3.common.CreateOutPath(MainFile=Project_GDB + '\\Int_'+str(year),appendix='EventTable',Extension='')
    arcpy.LocateFeaturesAlongRoutes_lr(
        in_features                = Project_GDB + '\\CreateInt_Final', 
        in_routes                = Routes[year], 
        route_id_field            = 'ROUTE_ID', 
        radius_or_tolerance        = '70 Feet', 
        out_table                = EventTable, 
        out_event_properties    = " ".join(['ROUTE_ID', "POINT", "MP"]),
        route_locations            = "ALL", 
        in_fields                = "FIELDS", 
        m_direction_offsetting    = "M_DIRECTON"
    )
    df = hsmpy3.common.AttributeTabletoDF(EventTable)

    df['SurfWid'] = 30.0
    df['AADT'] = 0.0
    df['FType'] = ''
    Att_DF = Att_Dict[2012]
    f = FloatProgress(min=0, max=df.shape[0])
    display(f)
    for i,r in df.iterrows():
        att_df = Att_DF[(Att_DF['ROUTE_ID']==r.ROUTE_ID) & (Att_DF.BEG_POINT<=r.MP) & (Att_DF.END_POINT>=r.MP)]
        if att_df.shape[0]>0:
            df.set_value(i,'SurfWid',max(list(att_df.SurfWid)))
            df.set_value(i,'AADT',max(list(att_df.AADT)))
            df.set_value(i,'FType',list(att_df.FType)[0])
        f.value += 1
    
    df['fpM'] = 0.0
    df['lpM'] = 0.0
    Att_DF = Att_Dict[year]
    Routes_DF = hsmpy3.common.AttributeTabletoDF(Routes[year],True)
    f = FloatProgress(min=0, max=df.shape[0])
    display(f)
    for i,r in df.iterrows():
        Rdf = Routes_DF[(Routes_DF['ROUTE_ID']==r.ROUTE_ID) & (Routes_DF['BEG_POINT']<=r.MP) & (Routes_DF['END_POINT']>=r.MP)]
        M = [s.M for s in list(Rdf.Shape)[0][0]]
        df.set_value(i,'fpM',M[0])
        df.set_value(i,'lpM',M[-1])
        f.value += 1    
        
    df['Major'] = 0
    intL = list(set(df.Int_ID))
    f = FloatProgress(min=0, max=len(intL))
    display(f)
    for intid in intL:
        Rdf = df[df.Int_ID==intid]
        maxAADT = max(Rdf.AADT)
        i = list(Rdf[Rdf.AADT==maxAADT].index)[0]
        rid = Rdf.loc[i,'ROUTE_ID']
        ridL = [rid,OppositeDirID(rid)]
        for ind in list(Rdf.index):
            if Rdf.loc[ind,'ROUTE_ID'] in ridL:
                df.set_value(ind,'Major',1)
            else:
                df.set_value(ind,'Major',0)
        f.value += 1
        
    # Create Intx App Event Table
    App_DF = df
    Routes_DF = Route_Dict[year]
    App_DF['Int_ID2'] = App_DF.Int_ID
    App_DF['RID'] = App_DF.ROUTE_ID
    print(strftime("%Y-%m-%d %H:%M:%S"))
    f = FloatProgress(min=0, max=Routes_DF.shape[0])
    display(f)
    App_Tab = pd.DataFrame(columns=['RID','BMP','EMP','IntProx','Int_ID2','Major'])
    x = 0
    for i,r in Routes_DF.iterrows():
        rid = r.ROUTE_ID
        app_all = App_DF[(App_DF.RID==rid) & (App_DF.Int_ID2!=0) & (~pd.isnull(App_DF.MP))]
        mp_List = list(set(app_all.MP))
        mp_List.sort()
        int_id_List = [list(app_all[app_all.MP==mp]['Int_ID2'])[0] for mp in mp_List]
        for int_id,mp in zip(int_id_List,mp_List):
            app    = App_DF[(App_DF.RID==rid) & (App_DF.Int_ID2==int_id)]
            app_x  = App_DF[(App_DF.Int_ID2==int_id) & (App_DF.RID!=rid)]

            int_w = [40.0]
            int_w.extend(list(app_x.SurfWid))
            int_w = float(max(int_w))/5280.0

            int_b = 250.0/5280.0

            if (1 in list(app.Signal)) and 1 in list(app.Major):
                int_b = 300.0/5280.0 + int_w
            if (1 in list(app.Signal)) and not (1 in list(app.Major)):
                int_b = 250.0/5280.0 + int_w
            if not (1 in list(app.Signal)) and 1 in list(app.Major):
                int_b = 250.0/5280.0 + int_w
            if not (1 in list(app.Signal)) and not (1 in list(app.Major)):
                int_b = 150.0/5280.0 + int_w

            bmp = max(mp - int_w - int_b,list(app.fpM)[0])
            emp = max(mp - int_w,list(app.fpM)[0])
            if emp>bmp:
                x += 1
                App_Tab.loc[x] = [rid,bmp,emp,1,int_id,{True:1,False:0}[1 in list(app.Major)]]

            bmp = max(mp - int_w,list(app.fpM)[0])
            emp = min(mp + int_w,list(app.lpM)[0])
            if emp>bmp:
                x += 1
                App_Tab.loc[x] = [rid,bmp,emp,2,int_id,{True:1,False:0}[1 in list(app.Major)]]

            bmp = min(mp + int_w,list(app.lpM)[0])
            emp = min(mp + int_w + int_b,list(app.lpM)[0])
            if emp>bmp:
                x += 1
                App_Tab.loc[x] = [rid,bmp,emp,1,int_id,{True:1,False:0}[1 in list(app.Major)]]
        f.value += 1
    App_Tab.to_csv('App_Tab{}.csv'.format(strftime("%Y%m%d_%H%M%S")))
    App_Tab       

    App_Tab = App_Tab.sort_values(by=['RID','BMP'])
    r = list(App_Tab.RID)[1:]
    r.append('')
    App_Tab['NextRID'] = r
    r = list(App_Tab.BMP)[1:]
    r.append('')
    App_Tab['NextBMP'] = r

    r = ['']
    r.extend(list(App_Tab.RID)[:-1])
    App_Tab['PrevRID'] = r
    r = ['']
    r.extend(list(App_Tab.EMP)[:-1])
    App_Tab['PrevEMP'] = r


    App_Tab['NewBMPEMP']  = [NewBMPEMP(rid,bmp,emp,nrid,nbmp,prid,pemp) for rid,bmp,emp,nrid,nbmp,prid,pemp in 
                             zip(App_Tab.RID,App_Tab.BMP,App_Tab.EMP,App_Tab.NextRID,App_Tab.NextBMP,App_Tab.PrevRID,App_Tab.PrevEMP)]

    App_Tab    
    
    f = FloatProgress(min=0, max=App_Tab.shape[0])
    display(f)
    App_Table = hsmpy3.common.CreateOutPath(Project_GDB + '\\CreateInt_AppTab',str(year),'')
    arcpy.management.CreateTable(out_name=os.path.basename(App_Table),out_path=Project_GDB)
    arcpy.management.AddField(App_Table,'RID','Text')
    arcpy.management.AddField(App_Table,'BMP','Double')
    arcpy.management.AddField(App_Table,'EMP','Double')
    arcpy.management.AddField(App_Table,'Int_ID','Long')
    arcpy.management.AddField(App_Table,'IntProx','Short')
    arcpy.management.AddField(App_Table,'Major','Short')
    ic = arcpy.InsertCursor(App_Table)
    for i,r in App_Tab.iterrows():
        row = ic.newRow()
        row.setValue('RID',r.RID)
        row.setValue('BMP',r.NewBMPEMP[0])
        row.setValue('EMP',r.NewBMPEMP[1])
        row.setValue('Int_ID',r.Int_ID2)
        row.setValue('Major',r.Major)
        row.setValue('IntProx',r.IntProx)
        ic.insertRow(row)
        f.value += 1
    del ic
    del row    
    IntAppOverlay = hsmpy3.common.CreateOutPath(MainFile=Project_GDB+'\\IntApp_Overlay',appendix=str(year),Extension='')
    arcpy.lr.OverlayRouteEvents(
            in_table                 = Project_GDB+'\\CreateInt_AppTab_' + str(year),
            in_event_properties      = ' '.join(['RID','LINE','BMP','EMP']), 
            overlay_table            = Project_GDB+'\\SegAtt_' + str(year),
            overlay_event_properties = ' '.join([RouteID,'LINE',BEG_STA,END_STA]),
            overlay_type             = "INTERSECT", 
            out_table                = IntAppOverlay, 
            out_event_properties     = ' '.join([RouteID,'LINE',BEG_STA,END_STA]),
            zero_length_events       = "ZERO", 
            in_fields                = "FIELDS", 
            build_index              = "INDEX"
    )
    
    SegIntAtt = hsmpy3.common.CreateOutPath(MainFile=Project_GDB+'\\SegInt_Att',appendix=str(year),Extension='')
    arcpy.lr.OverlayRouteEvents(
            in_table                 = Project_GDB+'\\SegAtt_' + str(year),
            in_event_properties      = ' '.join([RouteID,'LINE',BEG_STA,END_STA]), 
            overlay_table            = Project_GDB+'\\CreateInt_AppTab_' + str(year),
            overlay_event_properties = ' '.join(['RID','LINE','BMP','EMP']),
            overlay_type             = "UNION", 
            out_table                = SegIntAtt, 
            out_event_properties     = ' '.join([RouteID,'LINE',BEG_STA,END_STA]),
            zero_length_events       = "ZERO", 
            in_fields                = "FIELDS", 
            build_index              = "INDEX"
    )    

FloatProgress(value=0.0, max=323507.0)

FloatProgress(value=0.0, max=323507.0)

FloatProgress(value=0.0, max=103874.0)

2018-08-13 16:50:44


FloatProgress(value=0.0, max=212703.0)

KeyboardInterrupt: 

In [8]:
f.value

154756.0

In [ ]:
SegAtt_Dict = {year:hsmpy3.common.FCtoDF(Project_GDB+'\\SegInt_Att_' + str(year)) for year in Years}

In [ ]:
for year in Years:
    df = SegAtt_Dict[year]
    df = df[df.Int_ID>0]
    df = df.sort_values('Int_ID')
    Int = hsmpy3.common.CreateOutPath(MainFile=Project_GDB+'\\Int',Extension='',appendix=str(year))
    arcpy.conversion.FeatureClassToFeatureClass(in_features=Project_GDB+'\\CreateInt_Final',out_name=os.path.basename(Int),out_path=Project_GDB)
    arcpy.management.AddField(Int,'IFType','Text',10)
    arcpy.management.AddField(Int,'I_URBANCODE','Long')
    arcpy.management.AddField(Int,'FType_Major','Text',10)
    arcpy.management.AddField(Int,'FType_Minor','Text',10)
    arcpy.management.AddField(Int,'F_SYSTEM_Major','Short')
    arcpy.management.AddField(Int,'F_SYSTEM_Minor','Short')
    arcpy.management.AddField(Int,'OWNERSHIP_Major','Short')
    arcpy.management.AddField(Int,'OWNERSHIP_Minor','Short')
    arcpy.management.AddField(Int,'AADT_Major','Long')    
    arcpy.management.AddField(Int,'AADT_Minor','Long')    
    uc = arcpy.UpdateCursor(Int)
    f = FloatProgress(min=0, max=int(str(arcpy.management.GetCount(Int))),description=str(year))
    display(f)
    for row in uc:
        int_id = row.getValue('Int_ID')
        signal = row.getValue('Signal')
        legs = row.getValue('Legs')
        type_major = ''
        
        idf = df[df.Int_ID==int_id]

        ft = list(idf[idf.Major==1]['AADT'])
        if len(ft)>0:
            row.setValue('AADT_Major',int(max(ft)))

        ft = list(idf[idf.Major==0]['AADT'])
        if len(ft)>0:
            row.setValue('AADT_Minor',int(max(ft)))

        ft = [t for t in set(list(idf[idf.Major==1]['FType'])) if t]
        if len(ft)>0:
            row.setValue('FType_Major',ft[0]) 
            type_major = ft[0]
        ft = [t for t in set(list(idf[idf.Major==0]['FType'])) if t]
        if len(ft)>0:
            row.setValue('FType_Minor',ft[0]) 

        ft = [t for t in set(list(idf[idf.Major==1]['F_SYSTEM'])) if t]
        if len(ft)>0:
            row.setValue('F_SYSTEM_Major',int(ft[0])) 

        ft = [t for t in set(list(idf[idf.Major==0]['F_SYSTEM'])) if t]
        if len(ft)>0:
            row.setValue('F_SYSTEM_Minor',int(ft[0])) 

        ft = [t for t in set(list(idf[idf.Major==1]['OWNERSHIP'])) if t]
        if len(ft)>0:
            row.setValue('OWNERSHIP_Major',int(ft[0]))

        ft = [t for t in set(list(idf[idf.Major==0]['OWNERSHIP'])) if t]
        if len(ft)>0:
            row.setValue('OWNERSHIP_Minor',int(ft[0]))

        ft = [t for t in set(list(idf[idf.Major==1]['URBAN_CODE'])) if t]
        if len(ft)>0:
            row.setValue('I_URBANCODE',int(ft[0])) 
        
        row.setValue('IFType',FindIntPG(Type_Major=type_major,Legs=legs,Signal=signal))
        
        uc.updateRow(row)   
        f.value += 1
    del uc
    del row